# Four point functions in the $O(n)$ model

This notebook reproduces some known results about Potts and $O(n)$ four-point functions, using the new Julia code.

In [1]:
using BootstrapVirasoro, BootstrapVirasoro.LoopModels, BarnesDoubleGamma, BenchmarkTools
println("Number of threads: ", Threads.nthreads())

Number of threads: 64


# Bootstrap equations

We solve crossing symmetry equations for the spectrum of the $O(n)$ CFT:

\begin{align}
\mathcal{S}^{O(n)} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in 2\mathbb{N}+1} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \frac12\mathbb{N}^*\\ s\in\frac{1}{r}\mathbb{Z}}}  \ ,
\end{align}
We then numerically solve

\begin{align}
\sum_{V \in \mathcal{S}^{(s)}} D^{(s)}_V \mathcal I^{(s)}_V (x) = \sum_{V \in \mathcal{S}^{(t)}} D^{(t)}_V \mathcal I^{(t)}_V (x) = \sum_{V \in \mathcal{S}^{(u)}} D^{(u)}_V \mathcal I^{(u)}_V (x),
\end{align}

for some channel spectra $\mathcal{S}^{(s)}, \mathcal{S}^{(t)}, \mathcal{S}^{(u)}$, where $\mathcal I$ are interchiral conformal blocks, and the $D$'s are four-point structure constants.

We solve this system as
\begin{equation}
\underbrace{
\begin{bmatrix}
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [-\mathcal I^{(t)}_{V_j}(1-x_i)]_{ij} & [0] \\
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [0] & [-\mathcal I^{(u)}_{V_j}(1/x_i)]_{ij}
\end{bmatrix}}_A
\begin{bmatrix}
[D^{(s)}_{V_j}]_j \\
[D^{(t)}_{V_j}]_j \\
[D^{(u)}_{V_j}]_j
\end{bmatrix} = 
\begin{bmatrix}
\sum_{V_j \in \text{ known}} D^{(t)}_{V_j} \mathcal{I}^{(t)}_{V_j}(1-x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)\\
\sum_{V_j \in \text{ known}} D^{(u)}_{V_j} \mathcal{I}^{(u)}_{V_j}(1/x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)
\end{bmatrix}
\end{equation}
where the $x_i$ take more values than there are unknowns, i.e. $A$ is a tall rectangular matrix. 

To check numerical convergence, we solve two subsystems and compare the solutions. If the solutions are close, we know the computation has converged.

In [2]:
const Sig = Channels{Rational}

# Spectrum of O(n) model
c = CC(β=1 / (big"0.8" + big"0.2" * im))
ndiag_indices = [(r, s) for r in 1//2:1//2:20 for s in -1+1//(2r):1//(2r):1 if r * s % 1 == 0]
P = big"0.2" + big"0.1" * im
diag_field = Field(c, P=P)
fields = vcat([Field(c, r=r, s=s) for (r, s) in ndiag_indices], Field(c, P=P))

# Determine the parity of the number of legs in 4pt channels
function chan_parities(co::Correlation4)
    V1, V2, V3, V4 = co.fields
    chan_parities = Channels{Rational}((V1.r + V2.r) % 1, (V1.r + V4.r) % 1, (V1.r + V3.r) % 1)
end

# Compute series of blocks for any channel field compatible
# with the correlation co
function LoopSpectra(co, fields, fs)
    Vs = @channels filter(V -> V.r % 1 == chan_parities(co)[chan], fields)
    @channels ChannelSpectrum(co, chan, Vs[chan], fs[chan])
end

# Compute series of blocks, keeping only one of each (field, reflected field) pair
# given indices of the external fields.
function precompute_blocks(
    indices, fields=nothing;
    c=CC(β=1 / (big"0.8" + big"0.2" * im)),
    P=big"0.2" + big"0.1" * im,
    parity, precision=10
)
    setprecision(BigFloat, floor(Int, 1.2 * precision), base=10)
    Δmax = floor(Int, 1.5 * precision)
    if fields === nothing
        ndiag_indices = [(r, s) for r in 1//2:1//2:20 for s in -1+1//(2r):1//(2r):1 if r * s % 1 == 0]
        fields = vcat([Field(c, r=r, s=s) for (r, s) in ndiag_indices], Field(c, P=P))
    end
    co = Correlation([Field(c, r=r, s=s) for (r, s) in indices], Δmax)
    parity != 0 && (fields = filter(V -> V.diagonal || V.s >= 0, fields))
    fs = Channels(chan -> (V -> IBlock(co, chan, V, parity=parity)))
    LoopSpectra(co, fields, fs)
end

function compute_diagblocks(specs, Ps)
    co = specs.s.corr
    c = co.fields[1].c
    return @channels [IBlock(co, chan, Field(c, P=P)) for P in Ps[chan]]
end

# Solve crossing symmetry for given signature
# Optionnally for many different diagonal blocks in the channel
function solve(
    specs, signature;
    even_spin=Channels(false),
    rmax=3, show=true, Ps=nothing,
    fix=nothing,
    rels=nothing,
)
    specs = @channels filter(V -> V.diagonal && signature[chan] == 0 || !V.diagonal && V.r >= signature[chan], specs[chan])
    specs = @channels filter(V -> even_spin[chan] ? spin(V) % 2 == 0 : true, specs[chan])
    if Ps === nothing
        res = solve_bootstrap(specs, fix=fix, rels=rels)
        if show
            if rels === :stu
                printstyled("Channels s, t, u:\n", bold=true)
                Base.show(stdout, res.str_cst.s, rmax=rmax)
            else
                Base.show(stdout, res.str_cst, rmax=rmax)
            end
        end
    else
        diagblocks = compute_diagblocks(specs, Ps)
        res = solve_bootstrap(specs, diagblocks, fix=fix, rels=rels)
    end
    return res
end;

# Some numerical four-point functions

## $\langle (\frac{1}{2}, 0)^4 \rangle$

In [8]:
ind = ((1//2, 0), (1//2, 0), (1//2, 0), (1//2, 0))
blocks = precompute_blocks(ind, fields, parity=0, precision=20);

### Signature $(0, 1, 1)$

In [9]:
sol = solve(blocks, Sig(0, 1, 1), even_spin=Channels(true, false, false), rmax=2);

 Channel s
┌─────────────────┬───────────────────────────┬───────────┐
│ Field           │ Structure constant        │ Rel. err. │
├─────────────────┼───────────────────────────┼───────────┤
│ P = (0.2+0.1im) │ 1.0+0.0im                 │ 0         │
│ (1, 0)          │ -0.167173+0.134319im      │ 5e-18     │
│ (2, 0)          │ -0.00239852+0.000773346im │ 2.4e-16   │
│ (2, 1)          │ 0.000797319-0.000705013im │ 1.4e-16   │
└─────────────────┴───────────────────────────┴───────────┘
 Channel t
┌────────────┬───────────────────────────┬───────────┐
│ Field      │ Structure constant        │ Rel. err. │
├────────────┼───────────────────────────┼───────────┤
│ (1, 0)     │ 0.422102+0.0893706im      │ 1.8e-18   │
│ (1, 1)     │ 0.16568-0.0434621im       │ 8.9e-19   │
│ (2, 0)     │ 0.000999679-1.2823e-05im  │ 4.6e-16   │
│ (2, -1//2) │ 0.000592214+0.000185033im │ 1.1e-15   │
│ (2, 1//2)  │ 0.000592214+0.000185033im │ 9.7e-16   │
│ (2, 1)     │ 0.000344058+0.000230614im │ 6.7e-16   │
└──

In [6]:
# latex output renders in the notebook
display(c)
display(sol.correlation)
# latex copy-pastable output with backend=:latex keyword argument:
show(stdout, sol.str_cst, rmax=2, backend=:latex)

c = 1.614532871972318339100346020761245674740484429065743944636678200692041522491287 + 2.232249134948096885813148788927335640138408304498269896193771626297577854671372im, β = -1.176470588235294117647058823529411764705882352941176470588235294117647058823537 + 0.2941176470588235294117647058823529411764705882352941176470588235294117647058841im

NonChiralCorrelation{Complex{BigFloat}} with external fields
< (1//2, 0) (1//2, 0) (1//2, 0) (1//2, 0) >

 Channel s
\begin{tabular}{lll}
  \hline
  \textbf{Field} & \textbf{Structure constant} & \textbf{Rel. err.} \\\hline
  $P = (0.2+0.1im)$ & $1.0$ & 0 \\
  $\left(1,0\right)$ & $-0.16717252\;+\;0.13431905\mathrm{i}$ & 4.2e-18 \\
  $\left(2,0\right)$ & $-0.002398521\;+\;0.000773346\mathrm{i}$ & 1.8e-16 \\
  $\left(2,1\right)$ & $0.0007973192\;-\;0.00070501294\mathrm{i}$ & 1.2e-16 \\\hline
\end{tabular}
\\ Channel t
\begin{tabular}{lll}
  \hline
  \textbf{Field} & \textbf{Structure constant} & \textbf{Rel. err.} \\\hline
  $\left(1,0\right)$ & $0.42210183\;+\;0.08937056\mathrm{i}$ & 1.6e-18 \\
  $\left(1,1\right)$ & $0.16567983\;-\;0.04346215\mathrm{i}$ & 6.9e-19 \\
  $\left(2,0\right)$ & $0.0009996791\;-\;1.2822965e-5\mathrm{i}$ & 3.9e-16 \\
  $\left(2,\frac{-1}{2}\right)$ & $0.0005922143\;+\;0.00018503261\mathrm{i}$ & 8.3e-16 \\
  $\left(2,\frac{1}{2}\right)$ & $0.0005922143\;+\;0.00018503261\mathrm{i}$ & 9e-16 \\
  $\left(2,1\right)$ & $0.00034405786\;+\;0.00023061357\mathrm{i}$ & 6e-1

### Signature $(0, 1, 0)$, $D^{(s)}_{(2, 0)} = 0$

In [5]:
fix = [(:s, diag_field, 1), (:s, Field(c, r=2, s=0), 0)]
solve(blocks, Sig(0, 1, 0), fix=fix, rmax=2);

 Channel s
┌─────────────────┬──────────────────────────┬────────────────┐
│ Field           │ Structure constant       │ Relative error │
├─────────────────┼──────────────────────────┼────────────────┤
│ (P=0.10+0.10im) │                1.0+0.0im │            0.0 │
│          (1, 0) │      -1.16517-0.591022im │    7.65818e-15 │
│          (1, 1) │     0.464566+0.0798147im │     8.4386e-15 │
│          (2, 0) │                0.0+0.0im │            0.0 │
│       (2, 1//2) │ -0.00318409+0.00145081im │    1.01034e-14 │
│      (2, -1//2) │ -0.00318409+0.00145081im │     8.2106e-15 │
│          (2, 1) │  0.00257332-0.00274978im │    6.86828e-15 │
└─────────────────┴──────────────────────────┴────────────────┘
 Channel t
┌────────────┬──────────────────────────┬────────────────┐
│ Field      │ Structure constant       │ Relative error │
├────────────┼──────────────────────────┼────────────────┤
│     (1, 0) │     -0.433576-0.577531im │    1.38597e-14 │
│     (1, 1) │     0.672446+0.0319329i

### Solve for many different Ps

In [78]:
# solve for many different values of the momentum for the diag field in the channel.
# only recompute one column of the linear system each time: very cheap.
Ps = [big"0.8" + big"0.2"*im + i / big(100)*im for i in 1:3];
diagblocks = compute_diagblocks(blocks, Ps, :s)
sol_Ps = solve(blocks, Sig(0, 1, 1), even_spin=Channels(true, false, false),  Ps=Ps, diagchan=:s)
# print one of the results
show(stdout, sol_Ps[2].str_cst, rmax=2)

InterchiralBlock{Complex{BigFloat}}[G^(s)({ V_{P = 0.80000000000000000000000017 + 0.21000000000000000000000003im + n/β}, n ∈ -5:1:3}), G^(s)({ V_{P = 0.80000000000000000000000017 + 0.22000000000000000000000002im + n/β}, n ∈ -5:1:4}), G^(s)({ V_{P = 0.80000000000000000000000017 + 0.23000000000000000000000002im + n/β}, n ∈ -5:1:4})]
 Channel s
┌─────────────────┬─────────────────────────┬───────────┐
│ Field           │ Structure constant      │ Rel. err. │
├─────────────────┼─────────────────────────┼───────────┤
│ (1, 0)          │ 0.253897-0.354829im     │ 5.3e-18   │
│ (P=0.80+0.22im) │ 1.0+0.0im               │ 0         │
│ (2, 0)          │ 0.00488132-0.00418444im │ 2.1e-16   │
│ (2, 1)          │ -0.00118272+0.0024477im │ 1.2e-16   │
└─────────────────┴─────────────────────────┴───────────┘
 Channel t
┌────────────┬─────────────────────────┬───────────┐
│ Field      │ Structure constant      │ Rel. err. │
├────────────┼─────────────────────────┼───────────┤
│ (1, 0)     │ -1.0826

Benchmarks: on my 2015, 4-core Intel Macbook pro.
| Precision          | Python | Julia |
|----------|----------|----------|
| $\Delta_{\mathrm{max}}=20$, 13 digits  | 2min23s  | 2.3s  |
| $\Delta_{\mathrm{max}}=30$, 25 digits  | 8min10s  | 3.0s  |
| $\Delta_{\mathrm{max}}=40$, 35 digits  | 23min18s  | 12.6s  |

## $\left\langle P_1 P_2 P_3 P_4 \right\rangle$

In [7]:
ss = rand(Complex{BigFloat}, 4)
ind = Tuple((0, s) for s in ss)
blocks = precompute_blocks(ind, fields, parity=1, precision=15);

In [8]:
sol = solve(blocks, Sig(0, 0, 0), show=true);

 Channel s
┌─────────────────┬────────────────────────────┬────────────────┐
│ Field           │ Structure constant         │ Relative error │
├─────────────────┼────────────────────────────┼────────────────┤
│ (P=0.10+0.10im) │                  1.0+0.0im │            0.0 │
│          (1, 0) │      -0.043652+0.0461863im │    6.68646e-10 │
│          (1, 1) │     -0.00502903+0.098547im │     1.1386e-10 │
│          (2, 0) │   0.00079105-0.000735744im │     4.28782e-8 │
│       (2, 1//2) │  0.000110591-0.000183974im │     1.46143e-7 │
│          (2, 1) │ -0.000189192-3.69278e-06im │     5.65625e-8 │
│          (3, 0) │                          0 │      0.0607838 │
│       (3, 1//3) │                          0 │      0.0644803 │
│       (3, 2//3) │                          0 │      0.0288652 │
│          (3, 1) │                          0 │      0.0139439 │
└─────────────────┴────────────────────────────┴────────────────┘
 Channel t
┌─────────────────┬───────────────────────────┬───────

## $\langle (\frac{1}{2}, 0)^2 (1, 0)^2 \rangle$

In [9]:
ind = ((1//2, 0), (1//2, 0), (1, 0), (1, 0))
blocks = precompute_blocks(ind, fields2, parity=1, precision=13);

UndefVarError: UndefVarError: `fields2` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

### Signature $(0, \frac32, \frac32)$

In [10]:
sol = solve(blocks, Sig(0, 3//2, 3//2));

 Channel s
┌─────────────────┬────────────────────────┬────────────────┐
│ Field           │ Structure constant     │ Relative error │
├─────────────────┼────────────────────────┼────────────────┤
│ (P=0.10+0.10im) │              1.0+0.0im │            0.0 │
│          (1, 0) │    -1.79924+0.421555im │      0.0536187 │
│          (1, 1) │ 0.000273845+0.203333im │       0.179268 │
│          (2, 0) │      1.05837-2.37073im │       0.276612 │
│       (2, 1//2) │   -0.557097+0.386062im │        1.79628 │
│          (2, 1) │    -0.133632-0.76917im │       0.431461 │
│          (3, 0) │      41.9787-27.2899im │       0.626559 │
│       (3, 1//3) │     -34.9665+26.8837im │       0.680484 │
│       (3, 2//3) │       24.938-25.2727im │       0.667133 │
│          (3, 1) │     -11.1594+19.0543im │       0.716201 │
└─────────────────┴────────────────────────┴────────────────┘
 Channel t
┌───────────┬──────────────────────┬────────────────┐
│ Field     │ Structure constant   │ Relative error │
├─

### Signature $(1, \frac12, \frac32)$

In [11]:
solve(blocks, Sig(1, 1//2, 3//2), rmax=2);

 Channel s
┌───────────┬──────────────────────┬────────────────┐
│ Field     │ Structure constant   │ Relative error │
├───────────┼──────────────────────┼────────────────┤
│    (1, 0) │            1.0+0.0im │            0.0 │
│    (1, 1) │ -0.314676-0.227065im │        1.58253 │
│    (2, 0) │   -0.756645+5.0774im │       0.595367 │
│ (2, 1//2) │  -0.515941-3.00656im │       0.911785 │
│    (2, 1) │   0.973783+1.64767im │        1.26323 │
└───────────┴──────────────────────┴────────────────┘
 Channel t
┌───────────┬────────────────────────┬────────────────┐
│ Field     │ Structure constant     │ Relative error │
├───────────┼────────────────────────┼────────────────┤
│    (1, 0) │ -0.0288501+0.0165005im │          1.011 │
│    (1, 1) │  0.0329273-0.0770832im │       0.777737 │
│    (2, 0) │      3.39492+1.71516im │          2.017 │
│ (2, 1//2) │     -2.78334-1.07935im │        1.75393 │
│    (2, 1) │    1.69856+0.0715347im │        1.71933 │
└───────────┴────────────────────────┴──────

### Signature $(1, \frac32, \frac12)$

In [12]:
solve(blocks, Sig(1, 3//2, 1//2), rmax=2);

 Channel s
┌───────────┬──────────────────────┬────────────────┐
│ Field     │ Structure constant   │ Relative error │
├───────────┼──────────────────────┼────────────────┤
│    (1, 0) │            1.0+0.0im │            0.0 │
│    (1, 1) │ -0.422351-0.268128im │       0.215619 │
│    (2, 0) │    -2.42269+4.0059im │       0.383119 │
│ (2, 1//2) │    1.60603-2.13824im │       0.575963 │
│    (2, 1) │  -0.166614+1.56393im │       0.488147 │
└───────────┴──────────────────────┴────────────────┘
 Channel t
┌───────────┬────────────────────┬────────────────┐
│ Field     │ Structure constant │ Relative error │
├───────────┼────────────────────┼────────────────┤
│    (2, 0) │  2.02455+1.94047im │       0.673465 │
│ (2, 1//2) │ -1.66763-1.39314im │       0.676533 │
│    (2, 1) │ 1.12007+0.374722im │       0.710549 │
└───────────┴────────────────────┴────────────────┘
 Channel u
┌───────────┬───────────────────────┬────────────────┐
│ Field     │ Structure constant    │ Relative error │
├──────

## $\left\langle (0, \frac12)^3 P_{(1, 0)} \right\rangle$

In [3]:
ind = ((0, 1//2), (0, c.β^2), (0, 1//2), (0, 1//2))
blocks = precompute_blocks(ind, fields, parity=1, precision=20);
sol_diag = solve(blocks, Sig(0, 0, 0), even_spin=Channels(true), rmax=3);

 Channel s
┌─────────────────┬───────────────────────────┬───────────┐
│ Field           │ Structure constant        │ Rel. err. │
├─────────────────┼───────────────────────────┼───────────┤
│ (P=0.10+0.10im) │ 1.0+0.0im                 │ 0         │
│ (1, 0)          │ 1.35314+0.621346im        │ 3.2e-18   │
│ (2, 0)          │ -0.0146027+0.00422597im   │ 1.9e-16   │
│ (2, 1)          │ -9.09864e-05+0.00247326im │ 1.3e-16   │
│ (3, 0)          │ 1.58764e-06+1.66872e-06im │ 1.9e-12   │
│ (3, 2//3)       │ 1.18639e-06-4.9787e-07im  │ 1.8e-12   │
└─────────────────┴───────────────────────────┴───────────┘
 Channel t
┌─────────────────┬───────────────────────────┬───────────┐
│ Field           │ Structure constant        │ Rel. err. │
├─────────────────┼───────────────────────────┼───────────┤
│ (P=0.10+0.10im) │ 1.0-8.82188e-20im         │ 3.4e-19   │
│ (1, 0)          │ 1.35314+0.621346im        │ 3e-18     │
│ (2, 0)          │ -0.0146027+0.00422597im   │ 9.6e-17   │
│ (2, 1)          

## $\left\langle (0, \frac12)^3 (1, 0) \right\rangle$

In [4]:
ind = ((0, 1//2), (1, 0), (0, 1//2), (0, 1//2))
blocks = precompute_blocks(ind, fields, parity=1, precision=20);

In [5]:
sol1 = solve(blocks, Sig(0, 1, 1), even_spin=Channels(true, false, false),show=true, rmax=2);
sol2 = solve(blocks, Sig(1, 0, 1), even_spin=Channels(false, true, false), show=false);
sol3 = solve(blocks, Sig(1, 1, 0), even_spin=Channels(false, false, true), show=false);

 Channel s
┌─────────────────┬───────────────────────────┬───────────┐
│ Field           │ Structure constant        │ Rel. err. │
├─────────────────┼───────────────────────────┼───────────┤
│ (P=0.10+0.10im) │ 1.0+0.0im                 │ 0         │
│ (1, 0)          │ 0                         │ 0.31      │
│ (2, 0)          │ -0.00585115-0.000436412im │ 1.5e-17   │
│ (2, 1)          │ 0.00227685-0.000981606im  │ 1.5e-17   │
└─────────────────┴───────────────────────────┴───────────┘
 Channel t
┌───────────┬──────────────────────────┬───────────┐
│ Field     │ Structure constant       │ Rel. err. │
├───────────┼──────────────────────────┼───────────┤
│ (1, 0)    │ 0.676571+0.310673im      │ 8.6e-20   │
│ (1, 1)    │ 0.286591-0.0446828im     │ 4.8e-20   │
│ (2, 0)    │ -0.00437577+0.00233119im │ 1e-17     │
│ (2, 1//2) │ -0.00327768+0.00228679im │ 9.9e-18   │
│ (2, 1)    │ -0.00118392+0.00172743im │ 8.3e-18   │
└───────────┴──────────────────────────┴───────────┘
 Channel u
┌─────────

In [6]:
show(
    stdout,
    sol1.str_cst.s + sol2.str_cst.s + sol3.str_cst.s - sol_diag.str_cst.s,
    rmax = 2
)

┌─────────────────┬────────────────────────────┬───────────┐
│ Field           │ Structure constant         │ Rel. err. │
├─────────────────┼────────────────────────────┼───────────┤
│ (P=0.10+0.10im) │ 0.0+0.0im                  │ 0         │
│ (1, 0)          │ 0                          │ 0.31      │
│ (1, 1)          │ 5.17633e-20-1.52054e-19im  │ 6e-19     │
│ (2, 0)          │ -2.60874e-19-4.11794e-18im │ 4.3e-16   │
│ (2, 1//2)       │ 1.18137e-18+1.02885e-18im  │ 2.7e-16   │
│ (2, 1)          │ -6.71868e-19-5.58104e-19im │ 4e-16     │
└─────────────────┴────────────────────────────┴───────────┘


## $\left\langle (0, \frac12)^3 (2, 0) \right\rangle$

In [16]:
ind = ((0, 1//2), (2, 0), (0, 1//2), (0, 1//2))
blocks = precompute_blocks(ind, fields, parity=1, precision=20);
solve(blocks, Sig(1, 2, 1), even_spin=Channels(false, true, false), rmax=3);

 Channel s
┌───────────┬────────────────────────────┬────────────────┐
│ Field     │ Structure constant         │ Relative error │
├───────────┼────────────────────────────┼────────────────┤
│    (1, 0) │                  1.0+0.0im │            0.0 │
│    (1, 1) │        -0.63854+0.181026im │    2.98082e-17 │
│    (2, 0) │                          0 │       0.880898 │
│ (2, 1//2) │                          0 │       0.970237 │
│    (2, 1) │                          0 │        1.06903 │
│    (3, 0) │ -7.86521e-05-0.000322813im │    9.03894e-12 │
│ (3, 1//3) │   4.37238e-05+0.00014278im │    1.86741e-11 │
│ (3, 2//3) │  4.92031e-05+9.64753e-05im │    1.78644e-11 │
│    (3, 1) │ -8.77471e-05-9.14424e-05im │    9.44656e-12 │
└───────────┴────────────────────────────┴────────────────┘
 Channel t
┌───────────┬────────────────────────┬────────────────┐
│ Field     │ Structure constant     │ Relative error │
├───────────┼────────────────────────┼────────────────┤
│    (2, 0) │    0.413186+0.31

## $\left\langle (0, \frac12)^3 (3, 0) \right\rangle$

# Polynomials

In [ ]:
import BootstrapVirasoro.LoopModels: compute_reference, ρ_residue

wrs(r, s, β) = 2cospi(r * β^2 - s)

βs = [big"0.74421" + (i-1)//300 + 0im for i in 1:20] # range of βs such that ns are between 0 and 1.
cs = [CC(β=β) for β in βs]
ns = [-2cospi(β^2) for β in βs]
DGs = [DoubleGamma(c.β) for c in cs]
inds = [((0, 1 // 2), (0, β^2), (0, 1 // 2), (0, 1 // 2)) for β in βs]
Ps = [[(big"0.2532" + i // 60) * cs[1].β / cs[j].β for i in 1:10] for j = eachindex(cs)]; # diag field identical in each channel
blocks = [[precompute_blocks(inds[j], c=c, P=P, parity=1, precision=25) for (i, P) = enumerate(Ps[j])] for (j, c) in enumerate(cs)]
sols = [[solve(blocks[j][i], Sig(0, 0, 0), even_spin=Channels(true), show=false, rels=:stu) for i = eachindex(Ps[1])] for j = eachindex(cs)];

In [27]:
function reduce_to_polyterm(r, s)
    vals = [[sols[j][i].str_cst.s.constants[Field(c, r=r, s=s)] for i = eachindex(Ps[1])] for (j, c) = enumerate(cs)]
    Threads.@threads for j = 1:length(cs)
        c = cs[j]
        DG = DGs[j]
        V = Field(c, r=r, s=s)
        n = -2cospi(c.β^2)
        Threads.@threads for i = 1:length(Ps[j])
            P = Ps[j][i]
            co = sols[j][i].correlation
            diag_field = Field(c, P=P)
            w = 2cospi(2c.β * P)
            DP = compute_reference(co, diag_field, :s, DG)
            DV = compute_reference(co, V, :s, DG)
            vals[j][i] *= DP / DV
            vals[j][i] -= ρ_residue(V, co.fields...) / (w - wrs(V.r, V.s, c.β))
        end
    end
    return [((2cospi(2c.β * Ps[j][i]), -2cospi(c.β^2)), vals[j][i]) for i = eachindex(Ps[1]) for (j, c) = enumerate(cs)]
end

function fit_polynomial(data, varnames, degs)
    if varnames === (:w, :n) || varnames === (:n, :w)
        to_fit = data
    elseif varnames === (:w,)
        to_fit = [((d[1][1],), d[2]) for d in data[1:length(cs):end]]
    elseif varnames === (:n,)
        to_fit = [((d[1][2],), d[2]) for d in data[1:length(cs)]]
    else
        error("Unsupported varnames")
    end
    fit!(Polyfit(varnames, degs), to_fit)
end

fit_polynomial (generic function with 1 method)

## $d_{(r, s)}$ for $(r, s) = (1, 0), (2, 0), (2, 1)$

In [28]:
# only run to reset the data Dict! a bit expensive to recompute.
data = Dict()

Dict{Any, Any}()

In [29]:
for (r, s) in [(1, 0), (2, 0), (2, 1)]
    data[r, s] = reduce_to_polyterm(r, s)
end;

In [30]:
degs_w = [1, 2]
degs_n = [0, 2]

for (r, s) in [(1, 0), (2, 0), (2, 1)]
    display(fit_polynomial(data[r, s], (:w, :n), (degs_w[r], degs_n[r])))
end

(2) * w


(8) *  + (-4) * w + (-4) * w^(2) + (-2) * n^(2) + (1) * w*n^(2) + (1) * w^(2)*n^(2)


(8) *  + (4) * w + (-4) * w^(2) + (-2) * n^(2) + (-1) * w*n^(2) + (1) * w^(2)*n^(2)


In [31]:
data[3, 0] = reduce_to_polyterm(3, 0);
data[3, 2//3]  = reduce_to_polyterm(3, 2//3);

In [32]:
pf = fit_polynomial([(d[1], 3*d[2]) for d in data[3, 2//3]], (:w, :n), (3, 6))
println(pf)

(-6) * w^(3) + (-12) * w*n^(2) + (14) * w^(3)*n^(2) + (16) * w*n^(4) + (-10) * w^(3)*n^(4) + (-4) * w*n^(6) + (2) * w^(3)*n^(6)



In [33]:
pf = fit_polynomial([(d[1], 3*d[2]) for d in data[3,0]], (:w, :n), (3, 6))
println(pf)

(32) * w^(3)*n^(2) + (-8) * w*n^(4) + (-16) * w^(3)*n^(4) + (2) * w*n^(6) + (2) * w^(3)*n^(6)



In [34]:
for s in [0, 1//2, 1]
    data[4, s] = reduce_to_polyterm(4, s)
end

In [35]:
pf = fit_polynomial([(d[1], 4*d[2]) for d in data[4,0]], (:w, :n), (4, 10))

(-512.18-6.4642e-07I) *  + (-255.91-8.894e-07I) * w + (255.95+5.5588e-07I) * w^(2) + (8.937e-08-5.1841e-08I) * w^(3) + (0.045217-2.7207e-09I) * w^(4) + (3.5283+1.1462e-05I) * n + (-1.7641+1.5538e-05I) * w*n + (0.88208-9.7726e-06I) * w^(2)*n + (-1.565e-06+9.1947e-07I) * w^(3)*n + (-0.88208+4.7102e-08I) * w^(4)*n + (1761.2-9.0196e-05I) * n^(2) + (527.42-0.00012051I) * w*n^(2) + (-519.71+7.6265e-05I) * w^(2)*n^(2) + (1.2166e-05-7.2375e-06I) * w^(3)*n^(2) + (-120.29-3.6201e-07I) * w^(4)*n^(2) + (159.37+0.00041484I) * n^(3) + (-79.685+0.00054651I) * w*n^(3) + (39.843-0.00034797I) * w^(2)*n^(3) + (-5.5302e-05+3.3298e-05I) * w^(3)*n^(3) + (-39.843+1.6269e-06I) * w^(4)*n^(3) + (-3004.5-0.0012352I) * n^(4) + (126.25-0.0016053I) * w*n^(4) + (40.873+0.0010282I) * w^(2)*n^(4) + (0.00016281-9.9177e-05I) * w^(3)*n^(4) + (487.13-4.7361e-06I) * w^(4)*n^(4) + (1261.6+0.0024885I) * n^(5) + (-630.79+0.0031923I) * w*n^(5) + (315.4-0.0020564I) * w^(2)*n^(5) + (-0.00032447+0.00019988I) * w^(3)*n^(5) + (-315

In [36]:
pf = fit_polynomial([(d[1], 4*d[2]) for d in data[4,0]], (:w, :n), (4, 11))

(-511.98-5.6114e-06I) *  + (-256.01-3.8036e-06I) * w + (256+3.2737e-06I) * w^(2) + (4.3944e-07-4.1252e-07I) * w^(3) + (-0.0039192-6.235e-09I) * w^(4) + (-0.3356+0.00010907I) * n + (0.16792+7.2827e-05I) * w*n + (-0.083904-6.3201e-05I) * w^(2)*n + (-8.4469e-06+8.01e-06I) * w^(3)*n + (0.083877+1.1619e-07I) * w^(4)*n + (1795.3-0.0009515I) * n^(2) + (510.37-0.00062604I) * w*n^(2) + (-511.19+0.00054774I) * w^(2)*n^(2) + (7.2895e-05-6.9807e-05I) * w^(3)*n^(2) + (-128.81-9.7165e-07I) * w^(4)*n^(2) + (-18.923+0.0049187I) * n^(3) + (9.4666+0.00319I) * w*n^(3) + (-4.7309-0.0028133I) * w^(2)*n^(3) + (-0.00037286+0.00036048I) * w^(3)*n^(3) + (4.7296+4.8148e-06I) * w^(4)*n^(3) + (-2390.6-0.016744I) * n^(4) + (-180.74-0.010708I) * w*n^(4) + (194.36+0.0095176I) * w^(2)*n^(4) + (0.0012563-0.0012258I) * w^(3)*n^(4) + (333.64-1.5713e-05I) * w^(4)*n^(4) + (-200.49+0.039422I) * n^(5) + (100.29+0.02487I) * w*n^(5) + (-50.124-0.022273I) * w^(2)*n^(5) + (-0.0029285+0.0028829I) * w^(3)*n^(5) + (50.114+3.5477e-

In [45]:
pf = fit_polynomial([(d[1], 4*d[2]) for d in data[4,0]], (:w, :n), (4, 12))
show(stdout, pf, cutoff=1e-1)

(-512) *  + (-256) * w + (256) * w^2 + (1792) * n^2 + (511.99) * w*n^2 + (-512) * w^2*n^2 + (-128) * w^4*n^2 + (-2463.9-0.14369I) * n^4 + (-144.2) * w*n^4 + (176.06) * w^2*n^4 + (352) * w^4*n^4 + (-0.19411+0.38599I) * n^5 + (0.54347+0.13615I) * w*n^5 + (-0.15227-0.16986I) * w^2*n^5 + (1696.4-0.7481I) * n^6 + (-265.05-0.25996I) * w*n^6 + (184.3+0.32749I) * w^2*n^6 + (-344.02) * w^4*n^6 + (-0.51659+1.0543I) * n^7 + (1.4804+0.3611I) * w*n^7 + (-0.42043-0.45919I) * w^2*n^7 + (-615.48-1.0725I) * n^8 + (194.5-0.36226I) * w*n^8 + (-129.57+0.46486I) * w^2*n^8 + (145.97) * w^4*n^8 + (-0.36725+0.76835I) * n^9 + (1.0764+0.25605I) * w*n^9 + (-0.30958-0.33146I) * w^2*n^9 + (112.17-0.36803I) * n^10 + (-48.515-0.12107I) * w*n^10 + (28.149+0.15805I) * w^2*n^10 + (-28.012) * w^4*n^10 + (0.10586I) * n^11 + (0.14801) * w*n^11 + (-7.9936) * n^12 + (3.9807) * w*n^12 + (-1.9943) * w^2*n^12 + (1.9995) * w^4*n^12


In [46]:
pf = fit_polynomial([(d[1], 4*d[2]) for d in data[4,0]], (:w, :n), (4, 13))
show(stdout, pf, cutoff=1e-1)

(-512) *  + (-256) * w + (256) * w^2 + (1792) * n^2 + (511.96) * w*n^2 + (-511.99) * w^2*n^2 + (-128) * w^4*n^2 + (0.15725I) * n^3 + (0.22374) * w*n^3 + (-2463.7-0.66003I) * n^4 + (-144.94-0.25466I) * w*n^4 + (176.26+0.29631I) * w^2*n^4 + (351.98) * w^4*n^4 + (-0.98107+1.9751I) * n^5 + (2.8098+0.76148I) * w*n^5 + (-0.77864-0.88613I) * w^2*n^5 + (-0.10503+0.12675I) * w^3*n^5 + (1698.1-4.336I) * n^6 + (-270.17-1.6719I) * w*n^6 + (185.71+1.9447I) * w^2*n^6 + (0.23063-0.27798I) * w^3*n^6 + (-344.14) * w^4*n^6 + (-3.4969+7.0725I) * n^7 + (10.063+2.7294I) * w*n^7 + (-2.7926-3.1718I) * w^2*n^7 + (-0.37648+0.45301I) * w^3*n^7 + (0.23178) * w^4*n^7 + (-611.77-8.5733I) * n^8 + (183.8-3.3139I) * w*n^8 + (-126.61+3.8457I) * w^2*n^8 + (0.45694-0.54869I) * w^3*n^8 + (145.72) * w^4*n^8 + (-3.7653+7.6301I) * n^9 + (10.862+2.9562I) * w*n^9 + (-3.0142-3.4243I) * w^2*n^9 + (-0.40737+0.48796I) * w^3*n^9 + (0.25061) * w^4*n^9 + (114.39-4.8473I) * n^10 + (-54.903-1.8837I) * w*n^10 + (29.915+2.177I) * w^2*n^

In [ ]:
fake_data = [
    ((w, n), (n^2-4)^2 * (n^2-1)^2 * (n^6 + 2n^5 + n^4 + 2n^2 - 1))
    for n in ns
        for w in ws
]

In [49]:
ns = -2cospi.(βs.^2)
ws = 2cospi.(2 * cs[1].β .* Ps[1])

7-element Vector{Complex{BigFloat}}:
 1.8072458062710616031853870109048 - 1.8469212555472491630302806134391im
 1.8729561580226561899306377776626 - 1.963534336340791904445761973145im
 1.9427624865157424864720482004534 - 2.0844414797253474978880602457096im
 2.0168174515425381987492391997798 - 2.209907098108575382216534477863im
 2.0952830042587297312673936039495 - 2.3402055728488182547812762420659im
  2.178330741355225811877794326862 - 2.4756218543010013075737836438511im
 2.2661422803237780288438700372051 - 2.6164520849759204219886765510383im

In [55]:
acos(0.4) / 2 / π / cs[1].β

0.2479206941357582927460351962905 - 0.0im

In [57]:
cos(2 * π * cs[1].β * 0.25)

0.39107009122037116707512392896586 - 0.0im

In [58]:
cs[1].β * 0.25

0.18605250000000000000000000000003 + 0.0im

In [72]:
Ps = [[(big"0.2532" + i // 60) * cs[1].β / cs[j].β for i in 1:9] for j = eachindex(cs)]; # diag field identical in each channel
ws = [[2cospi(2 * c.β * P) for P in Ps[i]] for (i, c) = enumerate(cs)]
ws[1]

9-element Vector{Complex{BigFloat}}:
    0.60801657560880482321951590340119 - 0.0im
    0.45783164443969564265004747653723 - 0.0im
    0.30486742098105317111338281516937 - 0.0im
    0.15005248300157519825262119075523 - 0.0im
 -0.0056733568653028590950391627303246 - 0.0im
   -0.16136475630591723141759170592285 - 0.0im
   -0.31607658207911437300784732360198 - 0.0im
   -0.46886964749980331535111662679009 - 0.0im
   -0.61881641382361093744951740454852 - 0.0im

7-element Vector{Complex{BigFloat}}:
 1.2626233410400300296294736083196 - 0.072606537688406224390265588571806im
 1.2667636885445617023070790702461 - 0.14537185895985473572940353089616im
 1.2736743275922347754885190758295 - 0.21845509464196174550849976361333im
 1.2833703710924928939971626692923 - 0.29201607081088212668903390149509im
 1.2958730233690418926554903364394 - 0.36621565831571649078701886804342im
 1.3112096265316862891658437632543 - 0.44121612458773719547752819868735im
 1.3294137202709194259594541898301 - 0.51718148850380655448410546812603im